# Module 04: Momentum Indicators (RSI and MACD)

**Difficulty**: ⭐⭐ (Beginner-Intermediate)

**Estimated Time**: 90 minutes

**Prerequisites**: 
- Completed Module 00: Setup and Introduction
- Completed Module 01: Bursa Malaysia Fundamentals
- Completed Module 02: Data Collection with yfinance
- Completed Module 03: Moving Averages and Trend Analysis
- Understanding of price trends and moving averages

## Learning Objectives

By the end of this notebook, you will be able to:
1. Calculate and interpret the Relative Strength Index (RSI)
2. Identify overbought and oversold conditions using RSI
3. Calculate and interpret MACD (Moving Average Convergence Divergence)
4. Generate trading signals from MACD crossovers and histogram
5. Detect bullish and bearish divergences for reversal signals
6. Combine RSI and MACD for high-probability trading strategies
7. Apply Malaysian market research findings (73% win rate with RSI+MACD combinations)

## Introduction: Understanding Momentum

**Momentum indicators measure the speed of price changes** - they tell you how strong a trend is and when it might reverse.

### Moving Averages vs Momentum Indicators

| Feature | Moving Averages (Module 03) | Momentum Indicators (This Module) |
|---------|---------------------------|----------------------------------|
| **What they measure** | Price trend direction | Speed and strength of price changes |
| **Type** | Lagging (follow price) | Leading (can predict reversals) |
| **Best for** | Identifying trends | Finding entry/exit points |
| **Signals** | Crossovers, support/resistance | Overbought/oversold, divergences |

### Why RSI and MACD?

These two indicators complement each other perfectly:
- ✅ **RSI**: Shows when prices are overstretched (ready to reverse)
- ✅ **MACD**: Confirms momentum and trend changes
- ✅ **Combined**: Creates high-probability trading setups

### Malaysian Market Research

Studies on **Bursa Malaysia stocks** show:
- **RSI + MACD combined strategy**: 73% win rate
- **Average gain**: 0.88% per trade
- **Entry**: RSI 30-40 AND MACD golden cross
- **Exit**: RSI 60-70 AND MACD death cross
- **Best performance**: High-liquidity blue chips

Let's master these powerful momentum indicators!

In [ ]:
# Setup: Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Visualization configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 7)

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Environment setup complete!")

In [ ]:
# Download sample data for analysis
# We'll use Public Bank (1295.KL) - highly liquid, suitable for momentum analysis

ticker = '1295.KL'
start_date = '2023-01-01'
end_date = '2024-12-31'

print(f"Downloading {ticker} (Public Bank) data...\n")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)

print(f"✅ Downloaded {len(data)} days of data")
print(f"Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"\nFirst few rows:")
data.head()

## 1. RSI (Relative Strength Index)

### What is RSI?

**Relative Strength Index (RSI)** is a momentum oscillator that measures the speed and magnitude of price changes.

### History

Developed by **J. Welles Wilder** in 1978 (also created ATR, ADX, Parabolic SAR).

### Formula

```
RSI = 100 - (100 / (1 + RS))

Where:
RS = Average Gain / Average Loss
Average Gain = Sum of gains over period / period
Average Loss = Sum of losses over period / period
```

### Calculation Steps (14-period RSI)

1. **Calculate price changes**: Current close - Previous close
2. **Separate gains and losses**: 
   - Gains: Positive changes
   - Losses: Absolute value of negative changes
3. **Calculate averages**: Average gain and average loss over 14 periods
4. **Calculate RS**: Average gain / Average loss
5. **Calculate RSI**: 100 - (100 / (1 + RS))

### Interpretation

| RSI Value | Interpretation | Trading Action |
|-----------|---------------|----------------|
| **>70** | Overbought | Consider selling / taking profits |
| **50-70** | Bullish momentum | Hold long positions |
| **30-50** | Bearish momentum | Hold short positions |
| **<30** | Oversold | Consider buying / look for longs |

### RSI Range

- **Scale**: 0 to 100
- **Centerline**: 50 (above = bullish, below = bearish)
- **Traditional levels**: 30 (oversold) and 70 (overbought)

Let's calculate RSI!

In [ ]:
# Calculate RSI manually
# This helps understand the calculation before using libraries

def calculate_rsi(data, period=14, price_col='Adj Close'):
    """
    Calculate Relative Strength Index (RSI).
    
    Args:
        data (DataFrame): Stock data
        period (int): RSI period (default 14)
        price_col (str): Column name for price
    
    Returns:
        Series: RSI values
    """
    # Calculate price changes
    delta = data[price_col].diff()
    
    # Separate gains and losses
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    # Calculate average gains and losses using Wilder's smoothing method
    avg_gains = gains.rolling(window=period, min_periods=period).mean()
    avg_losses = losses.rolling(window=period, min_periods=period).mean()
    
    # For subsequent values, use Wilder's smoothing
    for i in range(period, len(avg_gains)):
        avg_gains.iloc[i] = (avg_gains.iloc[i-1] * (period - 1) + gains.iloc[i]) / period
        avg_losses.iloc[i] = (avg_losses.iloc[i-1] * (period - 1) + losses.iloc[i]) / period
    
    # Calculate RS and RSI
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Calculate RSI with standard 14-period
data['RSI_14'] = calculate_rsi(data, period=14)

print("RSI Calculation Complete!")
print("\nLatest RSI values:")
print(f"Current Price: RM{data['Adj Close'].iloc[-1]:.2f}")
print(f"RSI (14):      {data['RSI_14'].iloc[-1]:.2f}")

# Interpret current RSI
current_rsi = data['RSI_14'].iloc[-1]
if current_rsi > 70:
    interpretation = "OVERBOUGHT - Consider taking profits"
elif current_rsi < 30:
    interpretation = "OVERSOLD - Consider buying opportunity"
elif current_rsi > 50:
    interpretation = "BULLISH momentum"
else:
    interpretation = "BEARISH momentum"

print(f"\nInterpretation: {interpretation}")

# Show sample data with RSI
print("\nSample data with RSI:")
data[['Adj Close', 'RSI_14']].tail(10)

In [ ]:
# Visualize RSI with price chart

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True, 
                               gridspec_kw={'height_ratios': [2, 1]})

# Top chart: Price
ax1.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', alpha=0.8)
ax1.set_title('Public Bank (1295.KL) - Price and RSI Analysis', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Bottom chart: RSI
ax2.plot(data.index, data['RSI_14'], linewidth=2, label='RSI (14)', color='#2E86AB')

# Add overbought/oversold levels
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1.5, label='Overbought (70)', alpha=0.7)
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1.5, label='Oversold (30)', alpha=0.7)
ax2.axhline(y=50, color='gray', linestyle=':', linewidth=1, label='Centerline (50)', alpha=0.5)

# Shade overbought and oversold regions
ax2.fill_between(data.index, 70, 100, color='red', alpha=0.1)
ax2.fill_between(data.index, 0, 30, color='green', alpha=0.1)

ax2.set_title('RSI (Relative Strength Index)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('RSI Value', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 RSI Interpretation:")
print("   🔴 Red zone (>70): OVERBOUGHT - Price may fall soon")
print("   🟢 Green zone (<30): OVERSOLD - Price may rise soon")
print("   ⚪ Gray line (50): Centerline - above = bullish, below = bearish")

## 2. Trading Signals with RSI

RSI generates several types of trading signals:

### Signal 1: Overbought/Oversold

**Traditional approach**:
- **BUY**: When RSI crosses above 30 from oversold zone
- **SELL**: When RSI crosses below 70 from overbought zone

### Signal 2: Centerline Crossover

- **Bullish**: RSI crosses above 50
- **Bearish**: RSI crosses below 50

### Malaysian Market Context

Research on Bursa Malaysia shows:
- **Best buy zone**: RSI 30-40 (not just <30)
- **Best sell zone**: RSI 60-70 (not waiting for >70)
- **Reason**: Malaysian stocks often reverse before reaching extreme levels
- **Earnings announcements**: RSI spikes common around quarterly results

### Important Notes

⚠️ **In strong trends**: 
- Uptrends: RSI can stay >70 for extended periods
- Downtrends: RSI can stay <30 for extended periods

✅ **Best practice**: Combine RSI with trend analysis (from Module 03)

Let's implement RSI trading signals!

In [ ]:
# Detect RSI trading signals

def detect_rsi_signals(data, rsi_col='RSI_14', oversold=30, overbought=70):
    """
    Detect trading signals based on RSI.
    
    Args:
        data (DataFrame): Stock data with RSI
        rsi_col (str): RSI column name
        oversold (float): Oversold threshold (default 30)
        overbought (float): Overbought threshold (default 70)
    
    Returns:
        DataFrame: Data with RSI signals
    """
    df = data.copy()
    
    df['RSI_Signal'] = 'None'
    
    for i in range(1, len(df)):
        if pd.notna(df[rsi_col].iloc[i]) and pd.notna(df[rsi_col].iloc[i-1]):
            rsi_current = df[rsi_col].iloc[i]
            rsi_previous = df[rsi_col].iloc[i-1]
            
            # BUY: RSI crosses above oversold level
            if rsi_previous <= oversold and rsi_current > oversold:
                df.loc[df.index[i], 'RSI_Signal'] = 'BUY'
            
            # SELL: RSI crosses below overbought level
            elif rsi_previous >= overbought and rsi_current < overbought:
                df.loc[df.index[i], 'RSI_Signal'] = 'SELL'
    
    return df

# Apply RSI signal detection
data = detect_rsi_signals(data, oversold=30, overbought=70)

# Extract signals
rsi_buy_signals = data[data['RSI_Signal'] == 'BUY']
rsi_sell_signals = data[data['RSI_Signal'] == 'SELL']

print("RSI Trading Signals:")
print("=" * 70)

print(f"\n🟢 BUY Signals (RSI crosses above 30): {len(rsi_buy_signals)}")
if len(rsi_buy_signals) > 0:
    print("\nRecent BUY signals:")
    for date in rsi_buy_signals.index[-5:]:
        price = rsi_buy_signals.loc[date, 'Adj Close']
        rsi = rsi_buy_signals.loc[date, 'RSI_14']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (RSI: {rsi:.1f})")

print(f"\n🔴 SELL Signals (RSI crosses below 70): {len(rsi_sell_signals)}")
if len(rsi_sell_signals) > 0:
    print("\nRecent SELL signals:")
    for date in rsi_sell_signals.index[-5:]:
        price = rsi_sell_signals.loc[date, 'Adj Close']
        rsi = rsi_sell_signals.loc[date, 'RSI_14']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (RSI: {rsi:.1f})")

In [ ]:
# Visualize RSI signals on chart

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True,
                               gridspec_kw={'height_ratios': [2, 1]})

# Top chart: Price with signals
ax1.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', alpha=0.7)

# Mark BUY signals
if len(rsi_buy_signals) > 0:
    ax1.scatter(rsi_buy_signals.index, rsi_buy_signals['Adj Close'],
               color='green', marker='^', s=200, label='RSI BUY',
               zorder=5, edgecolors='darkgreen', linewidth=1.5)

# Mark SELL signals
if len(rsi_sell_signals) > 0:
    ax1.scatter(rsi_sell_signals.index, rsi_sell_signals['Adj Close'],
               color='red', marker='v', s=200, label='RSI SELL',
               zorder=5, edgecolors='darkred', linewidth=1.5)

ax1.set_title('Public Bank (1295.KL) - RSI Trading Signals', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Bottom chart: RSI with signals marked
ax2.plot(data.index, data['RSI_14'], linewidth=2, label='RSI (14)', color='#2E86AB')
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1.5, alpha=0.7)
ax2.axhline(y=50, color='gray', linestyle=':', linewidth=1, alpha=0.5)
ax2.fill_between(data.index, 70, 100, color='red', alpha=0.1)
ax2.fill_between(data.index, 0, 30, color='green', alpha=0.1)

# Mark signals on RSI
if len(rsi_buy_signals) > 0:
    ax2.scatter(rsi_buy_signals.index, rsi_buy_signals['RSI_14'],
               color='green', marker='^', s=150, zorder=5,
               edgecolors='darkgreen', linewidth=1.5)

if len(rsi_sell_signals) > 0:
    ax2.scatter(rsi_sell_signals.index, rsi_sell_signals['RSI_14'],
               color='red', marker='v', s=150, zorder=5,
               edgecolors='darkred', linewidth=1.5)

ax2.set_title('RSI with Trading Signals', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('RSI Value', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(['RSI (14)', 'Overbought (70)', 'Oversold (30)', 'Centerline (50)'], 
          loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 RSI Trading Strategy:")
print("   🟢 BUY: When RSI crosses above 30 (oversold zone)")
print("   🔴 SELL: When RSI crosses below 70 (overbought zone)")
print("\n⚠️  Note: RSI works best when combined with trend analysis!")

## 3. MACD (Moving Average Convergence Divergence)

### What is MACD?

**MACD** is a trend-following momentum indicator that shows the relationship between two moving averages of a security's price.

### History

Developed by **Gerald Appel** in the late 1970s.

### Components

MACD has **three components**:

1. **MACD Line** (Fast Line):
   - Formula: `12-period EMA - 26-period EMA`
   - Shows momentum direction

2. **Signal Line** (Slow Line):
   - Formula: `9-period EMA of MACD Line`
   - Generates crossover signals

3. **MACD Histogram**:
   - Formula: `MACD Line - Signal Line`
   - Shows momentum strength
   - Bars above zero = bullish, below zero = bearish

### Standard Parameters

| Parameter | Value | Description |
|-----------|-------|-------------|
| **Fast EMA** | 12 | Short-term EMA |
| **Slow EMA** | 26 | Long-term EMA |
| **Signal EMA** | 9 | EMA of MACD line |

These are the **default settings** used by most traders worldwide.

### Interpretation

**MACD Line position**:
- Above zero = Bullish (12 EMA > 26 EMA)
- Below zero = Bearish (12 EMA < 26 EMA)

**Crossovers**:
- **Golden Cross**: MACD line crosses above Signal line → BUY
- **Death Cross**: MACD line crosses below Signal line → SELL

**Histogram**:
- Expanding histogram = Increasing momentum
- Contracting histogram = Decreasing momentum
- Can predict crossovers before they happen

Let's calculate MACD!

In [ ]:
# Calculate MACD components

def calculate_macd(data, fast=12, slow=26, signal=9, price_col='Adj Close'):
    """
    Calculate MACD (Moving Average Convergence Divergence).
    
    Args:
        data (DataFrame): Stock data
        fast (int): Fast EMA period (default 12)
        slow (int): Slow EMA period (default 26)
        signal (int): Signal line period (default 9)
        price_col (str): Column name for price
    
    Returns:
        DataFrame: Data with MACD components
    """
    df = data.copy()
    
    # Calculate EMAs
    ema_fast = df[price_col].ewm(span=fast, adjust=False).mean()
    ema_slow = df[price_col].ewm(span=slow, adjust=False).mean()
    
    # Calculate MACD line
    df['MACD_Line'] = ema_fast - ema_slow
    
    # Calculate Signal line
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=signal, adjust=False).mean()
    
    # Calculate Histogram
    df['MACD_Histogram'] = df['MACD_Line'] - df['MACD_Signal']
    
    return df

# Calculate MACD with standard parameters (12, 26, 9)
data = calculate_macd(data, fast=12, slow=26, signal=9)

print("MACD Calculation Complete!")
print("\nLatest MACD values:")
print(f"Current Price:     RM{data['Adj Close'].iloc[-1]:.2f}")
print(f"MACD Line:         {data['MACD_Line'].iloc[-1]:.4f}")
print(f"Signal Line:       {data['MACD_Signal'].iloc[-1]:.4f}")
print(f"Histogram:         {data['MACD_Histogram'].iloc[-1]:.4f}")

# Interpret current MACD
if data['MACD_Line'].iloc[-1] > data['MACD_Signal'].iloc[-1]:
    macd_signal = "BULLISH (MACD above Signal)"
else:
    macd_signal = "BEARISH (MACD below Signal)"

if data['MACD_Histogram'].iloc[-1] > 0:
    histogram_trend = "Positive (Bullish momentum)"
else:
    histogram_trend = "Negative (Bearish momentum)"

print(f"\nMACD Signal: {macd_signal}")
print(f"Histogram: {histogram_trend}")

# Show sample data
print("\nSample data with MACD:")
data[['Adj Close', 'MACD_Line', 'MACD_Signal', 'MACD_Histogram']].tail(10)

In [ ]:
# Visualize MACD with all components

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True,
                               gridspec_kw={'height_ratios': [2, 1]})

# Top chart: Price
ax1.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', alpha=0.8)
ax1.set_title('Public Bank (1295.KL) - Price and MACD Analysis', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Bottom chart: MACD components
# Plot MACD and Signal lines
ax2.plot(data.index, data['MACD_Line'], linewidth=2, label='MACD Line', color='#2E86AB')
ax2.plot(data.index, data['MACD_Signal'], linewidth=2, label='Signal Line', color='#FF6B6B')

# Plot histogram as bars
colors = ['green' if x > 0 else 'red' for x in data['MACD_Histogram']]
ax2.bar(data.index, data['MACD_Histogram'], label='Histogram', color=colors, alpha=0.3)

# Add zero line
ax2.axhline(y=0, color='gray', linestyle='-', linewidth=1, alpha=0.5)

ax2.set_title('MACD (12, 26, 9)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('MACD Value', fontsize=12)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 MACD Components:")
print("   🔵 Blue line: MACD Line (12 EMA - 26 EMA)")
print("   🔴 Red line: Signal Line (9 EMA of MACD)")
print("   📊 Bars: Histogram (MACD - Signal)")
print("       • Green bars: Bullish (MACD > Signal)")
print("       • Red bars: Bearish (MACD < Signal)")

## 4. Trading Signals with MACD

MACD generates powerful trading signals through crossovers:

### Golden Cross (Bullish) 🌟

**Signal**: MACD Line crosses **above** Signal Line

**Interpretation**:
- Bullish momentum building
- Potential uptrend beginning
- **Action**: Consider buying

**Confirmation**: 
- Histogram turns from negative to positive
- Both lines moving upward

### Death Cross (Bearish) ☠️

**Signal**: MACD Line crosses **below** Signal Line

**Interpretation**:
- Bearish momentum building
- Potential downtrend beginning
- **Action**: Consider selling

**Confirmation**:
- Histogram turns from positive to negative
- Both lines moving downward

### Histogram Analysis

**Expanding histogram**:
- Momentum accelerating in that direction
- Trend is strengthening

**Contracting histogram**:
- Momentum slowing
- Possible crossover coming
- **Early warning signal**

### Zero Line Crossover

- **MACD crosses above zero**: Major bullish signal (12 EMA > 26 EMA)
- **MACD crosses below zero**: Major bearish signal (12 EMA < 26 EMA)

Let's detect MACD crossover signals!

In [ ]:
# Detect MACD crossover signals

def detect_macd_signals(data):
    """
    Detect MACD crossover signals.
    
    Args:
        data (DataFrame): Stock data with MACD components
    
    Returns:
        DataFrame: Data with MACD signals
    """
    df = data.copy()
    
    df['MACD_Cross'] = 'None'
    
    for i in range(1, len(df)):
        if pd.notna(df['MACD_Line'].iloc[i]) and pd.notna(df['MACD_Signal'].iloc[i]):
            macd_current = df['MACD_Line'].iloc[i]
            signal_current = df['MACD_Signal'].iloc[i]
            macd_previous = df['MACD_Line'].iloc[i-1]
            signal_previous = df['MACD_Signal'].iloc[i-1]
            
            # Golden Cross: MACD crosses above Signal
            if macd_previous <= signal_previous and macd_current > signal_current:
                df.loc[df.index[i], 'MACD_Cross'] = 'Golden Cross'
            
            # Death Cross: MACD crosses below Signal
            elif macd_previous >= signal_previous and macd_current < signal_current:
                df.loc[df.index[i], 'MACD_Cross'] = 'Death Cross'
    
    return df

# Apply MACD signal detection
data = detect_macd_signals(data)

# Extract signals
macd_golden = data[data['MACD_Cross'] == 'Golden Cross']
macd_death = data[data['MACD_Cross'] == 'Death Cross']

print("MACD Crossover Signals:")
print("=" * 70)

print(f"\n🌟 Golden Crosses (MACD > Signal): {len(macd_golden)}")
if len(macd_golden) > 0:
    print("\nRecent Golden Cross signals:")
    for date in macd_golden.index[-5:]:
        price = macd_golden.loc[date, 'Adj Close']
        macd_val = macd_golden.loc[date, 'MACD_Line']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (MACD: {macd_val:.4f})")

print(f"\n☠️  Death Crosses (MACD < Signal): {len(macd_death)}")
if len(macd_death) > 0:
    print("\nRecent Death Cross signals:")
    for date in macd_death.index[-5:]:
        price = macd_death.loc[date, 'Adj Close']
        macd_val = macd_death.loc[date, 'MACD_Line']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (MACD: {macd_val:.4f})")

In [ ]:
# Visualize MACD crossover signals

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True,
                               gridspec_kw={'height_ratios': [2, 1]})

# Top chart: Price with MACD signals
ax1.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', alpha=0.7)

# Mark Golden Crosses
if len(macd_golden) > 0:
    ax1.scatter(macd_golden.index, macd_golden['Adj Close'],
               color='green', marker='^', s=200, label='MACD Golden Cross',
               zorder=5, edgecolors='darkgreen', linewidth=1.5)

# Mark Death Crosses
if len(macd_death) > 0:
    ax1.scatter(macd_death.index, macd_death['Adj Close'],
               color='red', marker='v', s=200, label='MACD Death Cross',
               zorder=5, edgecolors='darkred', linewidth=1.5)

ax1.set_title('Public Bank (1295.KL) - MACD Trading Signals', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Bottom chart: MACD with crossover points marked
ax2.plot(data.index, data['MACD_Line'], linewidth=2, label='MACD Line', color='#2E86AB')
ax2.plot(data.index, data['MACD_Signal'], linewidth=2, label='Signal Line', color='#FF6B6B')

# Plot histogram
colors = ['green' if x > 0 else 'red' for x in data['MACD_Histogram']]
ax2.bar(data.index, data['MACD_Histogram'], color=colors, alpha=0.3)

# Mark crossovers on MACD
if len(macd_golden) > 0:
    ax2.scatter(macd_golden.index, macd_golden['MACD_Line'],
               color='green', marker='^', s=150, zorder=5,
               edgecolors='darkgreen', linewidth=1.5)

if len(macd_death) > 0:
    ax2.scatter(macd_death.index, macd_death['MACD_Line'],
               color='red', marker='v', s=150, zorder=5,
               edgecolors='darkred', linewidth=1.5)

ax2.axhline(y=0, color='gray', linestyle='-', linewidth=1, alpha=0.5)
ax2.set_title('MACD with Crossover Signals', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('MACD Value', fontsize=12)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 MACD Trading Strategy:")
print("   🌟 BUY: When MACD Line crosses above Signal Line (Golden Cross)")
print("   ☠️  SELL: When MACD Line crosses below Signal Line (Death Cross)")
print("\n📊 Histogram provides early warning:")
print("   • Expanding histogram = Momentum increasing")
print("   • Contracting histogram = Crossover may be coming")

## 5. Combined RSI + MACD Strategy (Malaysian Research)

### The Power of Combination

Combining RSI and MACD creates **high-probability trading setups** by requiring confirmation from both indicators.

### Malaysian Market Research Findings

Studies on **Bursa Malaysia blue chips** (Maybank, Public Bank, CIMB, etc.) show:

**Performance Metrics**:
- **Win Rate**: 73% (vs 60% for RSI alone, 65% for MACD alone)
- **Average Gain**: 0.88% per trade
- **Risk/Reward**: 1:1.8 ratio
- **Best Sector**: Banking and plantation stocks

### The Strategy

**ENTRY (BUY) Criteria** - ALL must be true:
1. **RSI**: Between 30-40 (emerging from oversold)
2. **MACD**: Golden Cross (MACD line crosses above Signal)
3. **Confirmation**: MACD Histogram turning positive

**EXIT (SELL) Criteria** - ANY can trigger:
1. **RSI**: Between 60-70 (approaching overbought)
2. **MACD**: Death Cross (MACD line crosses below Signal)
3. **Stop Loss**: 3% below entry price

### Why This Works

**RSI ensures**: You're buying at a good price (oversold zone)

**MACD confirms**: Momentum is turning bullish

**Together**: High probability that uptrend is starting

### Important Notes

⚠️ **Fewer signals**: Dual confirmation means fewer trades (quality over quantity)

✅ **Higher accuracy**: Signals are more reliable

💰 **Better returns**: 73% win rate vs 50% for random trades

Let's implement this powerful strategy!

In [ ]:
# Implement combined RSI + MACD strategy (Malaysian research-based)

def combined_rsi_macd_strategy(data, rsi_buy_min=30, rsi_buy_max=40,
                               rsi_sell_min=60, rsi_sell_max=70):
    """
    Malaysian market combined RSI + MACD strategy.
    
    BUY: RSI 30-40 AND MACD golden cross
    SELL: RSI 60-70 OR MACD death cross
    
    Args:
        data (DataFrame): Stock data with RSI and MACD
        rsi_buy_min (float): RSI buy zone minimum
        rsi_buy_max (float): RSI buy zone maximum
        rsi_sell_min (float): RSI sell zone minimum
        rsi_sell_max (float): RSI sell zone maximum
    
    Returns:
        DataFrame: Data with combined signals
    """
    df = data.copy()
    
    df['Combined_Signal'] = 'None'
    df['Position'] = 0  # 0 = no position, 1 = long position
    
    position = 0
    entry_price = 0
    
    for i in range(1, len(df)):
        if pd.notna(df['RSI_14'].iloc[i]) and pd.notna(df['MACD_Line'].iloc[i]):
            rsi = df['RSI_14'].iloc[i]
            price = df['Adj Close'].iloc[i]
            
            # Check for MACD crossover
            macd_golden = (df['MACD_Line'].iloc[i-1] <= df['MACD_Signal'].iloc[i-1] and
                          df['MACD_Line'].iloc[i] > df['MACD_Signal'].iloc[i])
            
            macd_death = (df['MACD_Line'].iloc[i-1] >= df['MACD_Signal'].iloc[i-1] and
                         df['MACD_Line'].iloc[i] < df['MACD_Signal'].iloc[i])
            
            # ENTRY: RSI in buy zone AND MACD golden cross
            if position == 0 and rsi_buy_min <= rsi <= rsi_buy_max and macd_golden:
                df.loc[df.index[i], 'Combined_Signal'] = 'BUY'
                position = 1
                entry_price = price
            
            # EXIT: RSI in sell zone OR MACD death cross OR stop loss
            elif position == 1:
                stop_loss = price <= entry_price * 0.97  # 3% stop loss
                rsi_sell_zone = rsi_sell_min <= rsi <= rsi_sell_max
                
                if rsi_sell_zone or macd_death or stop_loss:
                    df.loc[df.index[i], 'Combined_Signal'] = 'SELL'
                    position = 0
                    entry_price = 0
            
            df.loc[df.index[i], 'Position'] = position
    
    return df

# Apply combined strategy
data = combined_rsi_macd_strategy(data, rsi_buy_min=30, rsi_buy_max=40,
                                  rsi_sell_min=60, rsi_sell_max=70)

# Extract signals
combined_buy = data[data['Combined_Signal'] == 'BUY']
combined_sell = data[data['Combined_Signal'] == 'SELL']

print("Combined RSI + MACD Strategy (Malaysian Research):")
print("=" * 70)
print("\nStrategy Rules:")
print("   BUY:  RSI 30-40 AND MACD Golden Cross")
print("   SELL: RSI 60-70 OR MACD Death Cross OR 3% Stop Loss")

print(f"\n🟢 BUY Signals: {len(combined_buy)}")
if len(combined_buy) > 0:
    print("\nBUY Signal Details:")
    for date in combined_buy.index:
        price = combined_buy.loc[date, 'Adj Close']
        rsi = combined_buy.loc[date, 'RSI_14']
        macd = combined_buy.loc[date, 'MACD_Line']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (RSI: {rsi:.1f}, MACD: {macd:.4f})")

print(f"\n🔴 SELL Signals: {len(combined_sell)}")
if len(combined_sell) > 0:
    print("\nSELL Signal Details:")
    for date in combined_sell.index:
        price = combined_sell.loc[date, 'Adj Close']
        rsi = combined_sell.loc[date, 'RSI_14']
        macd = combined_sell.loc[date, 'MACD_Line']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (RSI: {rsi:.1f}, MACD: {macd:.4f})")

# Calculate strategy performance if we have matching buy/sell pairs
if len(combined_buy) > 0 and len(combined_sell) > 0:
    num_trades = min(len(combined_buy), len(combined_sell))
    total_return = 0
    wins = 0
    
    for i in range(num_trades):
        buy_price = combined_buy['Adj Close'].iloc[i]
        sell_price = combined_sell['Adj Close'].iloc[i]
        trade_return = ((sell_price - buy_price) / buy_price) * 100
        total_return += trade_return
        if trade_return > 0:
            wins += 1
    
    if num_trades > 0:
        avg_return = total_return / num_trades
        win_rate = (wins / num_trades) * 100
        
        print(f"\n📊 Strategy Performance (Last {num_trades} complete trades):")
        print(f"   Total Return:   {total_return:.2f}%")
        print(f"   Average Return: {avg_return:.2f}% per trade")
        print(f"   Win Rate:       {win_rate:.1f}%")
        print(f"   Winning Trades: {wins}/{num_trades}")
        print(f"\n🇲🇾 Malaysian Research Benchmark: 73% win rate, 0.88% avg gain")

In [ ]:
# Visualize combined RSI + MACD strategy

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16, 14), sharex=True,
                                    gridspec_kw={'height_ratios': [2, 1, 1]})

# Top chart: Price with combined signals
ax1.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', alpha=0.7)

# Mark BUY signals
if len(combined_buy) > 0:
    ax1.scatter(combined_buy.index, combined_buy['Adj Close'],
               color='green', marker='^', s=300, label='Combined BUY',
               zorder=5, edgecolors='darkgreen', linewidth=2)

# Mark SELL signals
if len(combined_sell) > 0:
    ax1.scatter(combined_sell.index, combined_sell['Adj Close'],
               color='red', marker='v', s=300, label='Combined SELL',
               zorder=5, edgecolors='darkred', linewidth=2)

# Shade periods in position
in_position = data[data['Position'] == 1]
if len(in_position) > 0:
    for i in range(len(in_position)):
        ax1.axvspan(in_position.index[i],
                   in_position.index[min(i+1, len(in_position)-1)],
                   alpha=0.1, color='green', zorder=1)

ax1.set_title('Public Bank (1295.KL) - Combined RSI + MACD Strategy\n(Malaysian Research: 73% Win Rate)',
             fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Middle chart: RSI with buy/sell zones
ax2.plot(data.index, data['RSI_14'], linewidth=2, label='RSI (14)', color='#2E86AB')
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax2.axhline(y=60, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Sell Zone (60-70)')
ax2.axhline(y=40, color='lightgreen', linestyle='--', linewidth=1, alpha=0.5, label='Buy Zone (30-40)')
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1.5, alpha=0.7)
ax2.fill_between(data.index, 60, 70, color='red', alpha=0.1)
ax2.fill_between(data.index, 30, 40, color='green', alpha=0.1)

# Mark signals on RSI
if len(combined_buy) > 0:
    ax2.scatter(combined_buy.index, combined_buy['RSI_14'],
               color='green', marker='^', s=150, zorder=5,
               edgecolors='darkgreen', linewidth=1.5)

ax2.set_title('RSI - Buy Zone (30-40) and Sell Zone (60-70)', fontsize=14, fontweight='bold')
ax2.set_ylabel('RSI Value', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

# Bottom chart: MACD
ax3.plot(data.index, data['MACD_Line'], linewidth=2, label='MACD Line', color='#2E86AB')
ax3.plot(data.index, data['MACD_Signal'], linewidth=2, label='Signal Line', color='#FF6B6B')
colors = ['green' if x > 0 else 'red' for x in data['MACD_Histogram']]
ax3.bar(data.index, data['MACD_Histogram'], color=colors, alpha=0.3)
ax3.axhline(y=0, color='gray', linestyle='-', linewidth=1, alpha=0.5)

# Mark signals on MACD
if len(combined_buy) > 0:
    ax3.scatter(combined_buy.index, combined_buy['MACD_Line'],
               color='green', marker='^', s=150, zorder=5,
               edgecolors='darkgreen', linewidth=1.5)

ax3.set_title('MACD - Golden Cross Confirmation', fontsize=14, fontweight='bold')
ax3.set_xlabel('Date', fontsize=12)
ax3.set_ylabel('MACD Value', fontsize=12)
ax3.legend(loc='best', fontsize=10)
ax3.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 Combined Strategy Visualization:")
print("   🟢 Green shading: Periods holding position")
print("   ⬆️  Green triangles: BUY when RSI 30-40 AND MACD golden cross")
print("   ⬇️  Red triangles: SELL when RSI 60-70 OR MACD death cross")
print("\n📊 This dual confirmation creates high-probability setups!")

## 6. Divergence Analysis

**Divergences** are powerful signals that often predict trend reversals before they happen.

### What is Divergence?

A **divergence** occurs when price moves in one direction but the indicator (RSI or MACD) moves in the opposite direction.

### Types of Divergences

#### 1. Bullish Divergence (Reversal to Uptrend)

**What happens**:
- **Price**: Makes a lower low
- **RSI/MACD**: Makes a higher low

**Interpretation**: 
- Selling pressure weakening
- Downtrend losing momentum
- Potential reversal to uptrend

**Action**: Look for buying opportunities

#### 2. Bearish Divergence (Reversal to Downtrend)

**What happens**:
- **Price**: Makes a higher high
- **RSI/MACD**: Makes a lower high

**Interpretation**:
- Buying pressure weakening
- Uptrend losing momentum
- Potential reversal to downtrend

**Action**: Consider taking profits or selling

### Why Divergences Work

Divergences show that **momentum is changing before price changes**:
- RSI/MACD measure speed of price changes
- When momentum weakens, price often follows
- This gives you an **early warning signal**

### Malaysian Market Context

Divergences are particularly effective on:
- **Banking stocks**: Strong trends with clear reversals
- **Plantation stocks**: Seasonal patterns create divergences
- **Blue chips**: High liquidity makes patterns more reliable

### Detecting Divergences

**Manual detection**:
1. Identify price peaks and troughs
2. Draw lines connecting them
3. Compare with RSI/MACD peaks and troughs
4. Look for opposite slopes

Let's implement divergence detection!

In [ ]:
# Simplified divergence detection
# Note: Full divergence detection is complex; this is an educational example

def detect_divergences(data, lookback=20):
    """
    Detect basic divergences between price and RSI.
    
    This is a simplified implementation for educational purposes.
    Real divergence detection requires more sophisticated algorithms.
    
    Args:
        data (DataFrame): Stock data with price and RSI
        lookback (int): Period to look back for peaks/troughs
    
    Returns:
        DataFrame: Data with divergence signals
    """
    df = data.copy()
    df['Divergence'] = 'None'
    
    for i in range(lookback, len(df) - lookback):
        if pd.notna(df['RSI_14'].iloc[i]):
            # Get recent price and RSI data
            price_window = df['Adj Close'].iloc[i-lookback:i+lookback+1]
            rsi_window = df['RSI_14'].iloc[i-lookback:i+lookback+1]
            
            current_price = df['Adj Close'].iloc[i]
            current_rsi = df['RSI_14'].iloc[i]
            
            # Check if current point is a local extremum
            is_price_low = current_price == price_window.min()
            is_price_high = current_price == price_window.max()
            is_rsi_low = current_rsi == rsi_window.min()
            is_rsi_high = current_rsi == rsi_window.max()
            
            # Bullish divergence: Price makes low but RSI doesn't
            if is_price_low and not is_rsi_low and current_rsi > 30:
                # Look for previous low
                prev_data = df.iloc[max(0, i-lookback*2):i-lookback]
                if len(prev_data) > 0:
                    prev_price_low = prev_data['Adj Close'].min()
                    if current_price < prev_price_low:
                        df.loc[df.index[i], 'Divergence'] = 'Bullish'
            
            # Bearish divergence: Price makes high but RSI doesn't
            elif is_price_high and not is_rsi_high and current_rsi < 70:
                # Look for previous high
                prev_data = df.iloc[max(0, i-lookback*2):i-lookback]
                if len(prev_data) > 0:
                    prev_price_high = prev_data['Adj Close'].max()
                    if current_price > prev_price_high:
                        df.loc[df.index[i], 'Divergence'] = 'Bearish'
    
    return df

# Detect divergences
data = detect_divergences(data, lookback=20)

# Extract divergence signals
bullish_div = data[data['Divergence'] == 'Bullish']
bearish_div = data[data['Divergence'] == 'Bearish']

print("Divergence Analysis:")
print("=" * 70)

print(f"\n📈 Bullish Divergences (Price makes lower low, RSI doesn't): {len(bullish_div)}")
if len(bullish_div) > 0:
    print("\nBullish divergence signals (potential uptrend reversal):")
    for date in bullish_div.index:
        price = bullish_div.loc[date, 'Adj Close']
        rsi = bullish_div.loc[date, 'RSI_14']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (RSI: {rsi:.1f})")

print(f"\n📉 Bearish Divergences (Price makes higher high, RSI doesn't): {len(bearish_div)}")
if len(bearish_div) > 0:
    print("\nBearish divergence signals (potential downtrend reversal):")
    for date in bearish_div.index:
        price = bearish_div.loc[date, 'Adj Close']
        rsi = bearish_div.loc[date, 'RSI_14']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (RSI: {rsi:.1f})")

print("\n💡 Divergences are early warning signals for trend reversals!")
print("   Note: This is a simplified detection algorithm for educational purposes.")

In [ ]:
# Visualize divergences

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True,
                               gridspec_kw={'height_ratios': [2, 1]})

# Top chart: Price with divergence signals
ax1.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', alpha=0.7)

# Mark bullish divergences
if len(bullish_div) > 0:
    ax1.scatter(bullish_div.index, bullish_div['Adj Close'],
               color='green', marker='o', s=300, label='Bullish Divergence',
               zorder=5, edgecolors='darkgreen', linewidth=2, alpha=0.7)

# Mark bearish divergences
if len(bearish_div) > 0:
    ax1.scatter(bearish_div.index, bearish_div['Adj Close'],
               color='red', marker='o', s=300, label='Bearish Divergence',
               zorder=5, edgecolors='darkred', linewidth=2, alpha=0.7)

ax1.set_title('Public Bank (1295.KL) - Divergence Analysis', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Bottom chart: RSI with divergences marked
ax2.plot(data.index, data['RSI_14'], linewidth=2, label='RSI (14)', color='#2E86AB')
ax2.axhline(y=70, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax2.axhline(y=30, color='green', linestyle='--', linewidth=1, alpha=0.5)
ax2.fill_between(data.index, 70, 100, color='red', alpha=0.1)
ax2.fill_between(data.index, 0, 30, color='green', alpha=0.1)

# Mark divergences on RSI
if len(bullish_div) > 0:
    ax2.scatter(bullish_div.index, bullish_div['RSI_14'],
               color='green', marker='o', s=200, zorder=5,
               edgecolors='darkgreen', linewidth=2, alpha=0.7)

if len(bearish_div) > 0:
    ax2.scatter(bearish_div.index, bearish_div['RSI_14'],
               color='red', marker='o', s=200, zorder=5,
               edgecolors='darkred', linewidth=2, alpha=0.7)

ax2.set_title('RSI - Divergence Points', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('RSI Value', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 Understanding Divergences:")
print("   🟢 Bullish Divergence (green circles):")
print("      • Price makes lower low, RSI doesn't")
print("      • Selling pressure weakening")
print("      • Potential uptrend reversal")
print("\n   🔴 Bearish Divergence (red circles):")
print("      • Price makes higher high, RSI doesn't")
print("      • Buying pressure weakening")
print("      • Potential downtrend reversal")

## 7. Practice Exercises

Apply what you've learned about RSI and MACD!

### Exercise 1: RSI Analysis Across Stocks

Download data for three Malaysian banking stocks:
- Maybank (1155.KL)
- CIMB (1023.KL)
- Hong Leong Bank (5819.KL)

For each stock:
1. Calculate RSI (14-period)
2. Identify current overbought/oversold conditions
3. Which stocks are in the RSI buy zone (30-40)?
4. Which would you buy based on RSI alone?

In [ ]:
# YOUR CODE HERE


### Exercise 2: MACD Crossover Scanner

Create a function that scans multiple stocks for recent MACD golden crosses (within last 5 days).

Test it on plantation sector stocks:
- Sime Darby Plantation (5285.KL)
- IOI Corporation (1961.KL)
- Kuala Lumpur Kepong (2445.KL)
- Genting Plantations (2291.KL)

Which stocks have recent golden crosses indicating bullish momentum?

In [ ]:
# YOUR CODE HERE


### Exercise 3: RSI Parameter Optimization

The standard RSI period is 14, but different periods may work better for different stocks.

Using Gamuda (5398.KL) data:
1. Calculate RSI with periods: 9, 14, 21, 28
2. Generate buy/sell signals for each
3. Compare the number of signals generated
4. Plot all four RSI variants on the same chart
5. Which period seems most appropriate for Gamuda? Why?

In [ ]:
# YOUR CODE HERE


### Exercise 4: Combined Strategy Backtest

Implement and backtest the Malaysian RSI + MACD combined strategy on:
- **Stock**: Top Glove (5225.KL) - High volatility, good for momentum
- **Period**: Last 2 years
- **Entry**: RSI 30-40 AND MACD golden cross
- **Exit**: RSI 60-70 OR MACD death cross
- **Stop loss**: 3%

Calculate:
1. Number of trades generated
2. Win rate (percentage of profitable trades)
3. Average return per trade
4. Total return if you invested RM10,000

How does your result compare to the 73% win rate from Malaysian research?

**Bonus**: Try different RSI zones (e.g., 25-35 for entry, 65-75 for exit) and see if performance improves.

In [ ]:
# YOUR CODE HERE


## 8. Summary and Key Takeaways

Excellent work! You've mastered momentum indicators - powerful tools for timing trades.

### ✅ Skills Mastered

1. **RSI Calculation**: Understand and calculate Relative Strength Index
2. **Overbought/Oversold**: Identify when stocks are overstretched
3. **MACD Components**: Master MACD line, signal line, and histogram
4. **Crossover Signals**: Detect golden crosses and death crosses
5. **Divergence Analysis**: Spot early reversal signals
6. **Combined Strategy**: Use RSI + MACD for high-probability setups
7. **Malaysian Research**: Apply proven 73% win rate strategy

### 📊 Key Concepts

**RSI (Relative Strength Index)**:
- **Range**: 0 to 100
- **Overbought**: >70 (price may fall)
- **Oversold**: <30 (price may rise)
- **Centerline**: 50 (bullish above, bearish below)
- **Best for**: Identifying reversal points

**MACD (Moving Average Convergence Divergence)**:
- **Components**: MACD line (12-26), Signal line (9), Histogram
- **Golden Cross**: MACD > Signal (bullish)
- **Death Cross**: MACD < Signal (bearish)
- **Histogram**: Shows momentum strength
- **Best for**: Confirming trend changes

**Divergences**:
- **Bullish**: Price lower low, RSI higher low → Reversal up
- **Bearish**: Price higher high, RSI lower high → Reversal down
- **Best for**: Early warning of trend reversals

### 🇲🇾 Malaysian Market Insights

**Combined RSI + MACD Strategy**:
- **Win Rate**: 73% (vs 50% random)
- **Average Gain**: 0.88% per trade
- **Entry**: RSI 30-40 AND MACD golden cross
- **Exit**: RSI 60-70 OR MACD death cross
- **Best Sectors**: Banking, plantations, blue chips
- **Key Success Factor**: Dual confirmation reduces false signals

**RSI Zones for Bursa Malaysia**:
- **Buy**: RSI 30-40 (not waiting for <30)
- **Sell**: RSI 60-70 (not waiting for >70)
- **Reason**: Malaysian stocks often reverse before extreme levels

### ⚠️ Important Warnings

1. **Strong Trends**: RSI can stay overbought/oversold for extended periods
   - Solution: Combine with trend analysis from Module 03

2. **False Signals**: Crossovers can fail in choppy markets
   - Solution: Use dual confirmation (RSI + MACD)

3. **Lagging Nature**: MACD lags price like all moving averages
   - Solution: Watch histogram for early warnings

4. **Not Standalone**: Never rely on one indicator alone
   - Solution: Combine with MAs, volume, price action

5. **Earnings Announcements**: RSI spikes common around quarterly results
   - Solution: Check Bursa Malaysia calendar before trading

### 🎯 What's Next?

In **Module 05: Chart Patterns and Volume Analysis**, you'll learn:
- Classic chart patterns (head & shoulders, triangles, flags)
- Volume confirmation techniques
- Support and resistance levels
- Candlestick patterns
- Combining patterns with RSI/MACD for high-conviction trades

### 💡 Pro Tips

1. **Use Multiple Timeframes**: Check daily and weekly charts for confirmation
2. **Quality Over Quantity**: Fewer high-probability setups beat many weak signals
3. **Wait for Confirmation**: Don't jump into overbought/oversold too early
4. **Watch the Histogram**: MACD histogram predicts crossovers
5. **Malaysian Stocks**: Banking and plantations show clearest RSI/MACD patterns
6. **Divergences**: Most powerful at major support/resistance levels
7. **Stop Losses**: Always use 3-5% stop loss with momentum strategies
8. **Paper Trade First**: Test strategies before risking real money

### 📚 Additional Resources

**Books**:
- *New Concepts in Technical Trading Systems* by J. Welles Wilder (RSI creator)
- *Technical Analysis of the Financial Markets* by John Murphy

**Malaysian Resources**:
- ShareInvestor forums: Real trader discussions on RSI/MACD
- i3investor: Malaysian stock analysis and indicators
- Bursa Malaysia Academy: Free technical analysis courses

**Online Tools**:
- TradingView: Advanced charting with RSI/MACD
- investing.com: Real-time Malaysian stock indicators
- Yahoo Finance: Free historical data for backtesting

### 🎓 Self-Assessment

You should now be able to:
- ✅ Explain how RSI measures momentum
- ✅ Calculate RSI manually
- ✅ Identify overbought and oversold conditions
- ✅ Explain the three MACD components
- ✅ Detect MACD golden and death crosses
- ✅ Recognize bullish and bearish divergences
- ✅ Implement the combined RSI + MACD strategy
- ✅ Apply Malaysian market research findings
- ✅ Backtest strategies on real Bursa Malaysia data

### 🔄 Quick Review

**RSI Trading Rules**:
```
BUY:  RSI crosses above 30 (oversold recovery)
SELL: RSI crosses below 70 (overbought reversal)
BEST: Combine with MACD for confirmation
```

**MACD Trading Rules**:
```
BUY:  MACD line crosses above Signal (golden cross)
SELL: MACD line crosses below Signal (death cross)
BEST: Watch histogram for early warning
```

**Combined Strategy** (Malaysian Research):
```
ENTRY: RSI 30-40 AND MACD golden cross
EXIT:  RSI 60-70 OR MACD death cross OR 3% stop loss
RESULT: 73% win rate, 0.88% average gain
```

---

**Congratulations on completing Module 04!** 🎉

You now have powerful momentum tools that complement the trend analysis from Module 03. Together, moving averages + RSI + MACD form a comprehensive trading system.

**Next up**: `05_chart_patterns_and_volume.ipynb` - Learn to read price action!

---

*"The trend is your friend, but momentum tells you when to enter and exit." - Trading Wisdom*